In [1]:
# -*- coding: utf-8 -*
import os
import re
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
import warnings
from keras.utils import np_utils
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from keras.utils import to_categorical
from keras.backend import tensorflow_backend
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import StratifiedKFold
import scipy.linalg as LA
import cv2
import tensorflow as tf
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras import models, optimizers

/home/tsunei/anaconda3/envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
config = tf.ConfigProto(gpu_options = tf.GPUOptions(allow_growth = True))
session = tf.Session(config = config)
tensorflow_backend.set_session(session)

In [3]:
def list_pictures(directory, ext='jpg|jpeg|bmp|png|ppm'):
    return [os.path.join(root, f)
            for root, _, files in os.walk(directory) for f in files
            if re.match(r'([\w]+\.(?:' + ext + '))', f.lower())]

In [4]:
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_vgg16_trained_model.h5'
 
### データセットの準備
                                                    
image_size = 224
 
#X = []
#Y = []
X_eva = []
Y_eva = []
    
#print("人工物画像の教師データを取得してます...")
#for filepath in list_pictures('../data/CNN_data/test/j/'):
 #   warnings.filterwarnings('ignore')
  #  img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))        
   # X.append(img_eva)
    #Y.append(0) # 教師データ（正解）

#print("人工物画像の教師データを取得しました")

    
# 学習データの取得（非正解画像）
#print("自然物画像の教師データを取得してます...")
#for filepath in list_pictures('../data/CNN_data/test/s/'):
 #   img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))
  #  X.append(img_eva)
   # Y.append(1) # 教師データ（正解）

#print("自然物画像の教師データを取得しました")
    
# 学習データの取得（正解画像）
#print("人工物画像のテストデータを取得してます...")
#for filepath in list_pictures('../data/CNN_data/test/j_test/'):
 #   img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))        
  #  X_eva.append(img_eva)
   # Y_eva.append(0) # 教師データ（正解）

#print("人工物画像のテストデータを取得しました")

    
# 学習データの取得（非正解画像）
#print("自然物画像のテストデータを取得してます...")
#for filepath in list_pictures('../data/CNN_data/test/s_test/'):
 #   img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))
  #  X_eva.append(img_eva)
   # Y_eva.append(1) # 教師データ（正解）

#print("自然物画像のテストデータを取得しました")

print("人工物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/jinkou/'):
    warnings.filterwarnings('ignore')
    img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))        
    X_eva.append(img_eva)
    Y_eva.append(0) # 教師データ（正解）

print("人工物画像のテストデータを取得しました")

    
# 学習データの取得（非正解画像）
print("自然物画像のテストデータを取得してます...")
for filepath in list_pictures('../data/CNN_data/test/sizen/'):
    img_eva = img_to_array(load_img(filepath, target_size=(224, 224)))
    X_eva.append(img_eva)
    Y_eva.append(1) # 教師データ（正解）

print("自然物画像のテストデータを取得しました")

#X = np.array(X)   
#Y = np.array(Y)          
X_eva = np.array(X_eva)   
Y_eva = np.array(Y_eva)  
 
#img_gray = 0.299 * X[:, :, :, 0] + 0.587 * X[:, :, :, 1] + 0.114 * X[:, :, :, 2]
img_gray2 = 0.299 * X_eva[:, :, :, 0] + 0.587 * X_eva[:, :, :, 1] + 0.114 * X_eva[:, :, :, 2]

#X[:, :, :, 0] = img_gray[:]
#X[:, :, :, 1] = img_gray[:]
#X[:, :, :, 2] = img_gray[:]
X_eva[:, :, :, 0] = img_gray2[:]
X_eva[:, :, :, 1] = img_gray2[:]
X_eva[:, :, :, 2] = img_gray2[:]
    
#x_train = X.astype('float32')
x_test = X_eva.astype('float32')
#x_train /= 255
x_test /= 255

# ｙ　ラベルをワンホット表現に
#y_train = keras.utils.to_categorical(Y, num_classes)
#y_test = keras.utils.to_categorical(Y_eva, num_classes)

人工物画像のテストデータを取得してます...
人工物画像のテストデータを取得しました
自然物画像のテストデータを取得してます...
自然物画像のテストデータを取得しました


In [5]:
batch_size = 32
num_classes = 2
epochs = 100
ave = 0
fold_num = 3
seed = 5
np.random.seed(seed)
count = 0

kfold = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
jinkou_score = []
sizen_score = []
cvscores = []

for train, test in kfold.split(x_test, Y_eva):
    ### モデル構築 
    vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
    #vgg_conv.summary()
    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False

    model = models.Sequential()
    model.add(vgg_conv.layers[0])
    model.add(vgg_conv.layers[1])
    model.add(vgg_conv.layers[2])
    model.add(vgg_conv.layers[3])
    model.add(vgg_conv.layers[4])
    model.add(vgg_conv.layers[5])
    model.add(vgg_conv.layers[6])
    model.add(vgg_conv.layers[7])
    model.add(vgg_conv.layers[8])
    model.add(vgg_conv.layers[9])
    model.add(vgg_conv.layers[10])
    model.add(vgg_conv.layers[11])
    model.add(vgg_conv.layers[12])
    model.add(vgg_conv.layers[13])
    model.add(vgg_conv.layers[14])
    model.add(vgg_conv.layers[15])
    model.add(vgg_conv.layers[16])
    model.add(vgg_conv.layers[17])
    model.add(vgg_conv.layers[18])
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation='sigmoid'))   ### データは４種類

    #model.summary()

    model.compile(loss='binary_crossentropy',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['accuracy'])

    ### 学習
    model.fit(x_test[train], np_utils.to_categorical(Y_eva[train], 2), batch_size=batch_size, epochs=epochs, validation_data=(x_test[train], np_utils.to_categorical(Y_eva[train], 2)), shuffle=True)

    y_pred = model.predict_classes(x_test[test])

    sum_all_j = 0
    sum_all_s = 0
    sum_j = 0
    sum_s = 0

    for i in range(x_test[test].shape[0]):
        #print("%s:%s" % (y_pred[i], Y_eva[test][i]))
        if y_pred[i] == 1:
            if y_pred[i] == Y_eva[test][i]:           
                sum_j = sum_j + 1  

            sum_all_j = sum_all_j + 1  

        else:
            if y_pred[i] == Y_eva[test][i]:
                sum_s = sum_s + 1

            sum_all_s = sum_all_s + 1

            
    print("人工物画像の正解：",sum_j,"/",sum_all_j,"＝",sum_j / sum_all_j * 100,"%")
    print("自然物画像の正解：",sum_s,"/",sum_all_s,"＝",sum_s / sum_all_s * 100,"%")
    
    sum_j =  sum_j / sum_all_j * 100
    sum_s =  sum_s / sum_all_s * 100
    ave = (sum_s + sum_j) / 2 + ave

    #print("人工物画像の正解率＝",sum_j,"%")
    #print("自然物画像の正解率＝",sum_s,"%")
    #print("全体画像の正解率＝",(sum_s + sum_j) / 2,"%")
      
    # Evaluate
    scores = model.evaluate(x_test[test], keras.utils.to_categorical(Y_eva[test], num_classes), verbose=0)
    #print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    #cvscores.append(scores[1] * 100)
    jinkou_score.append(sum_j)
    sizen_score.append(sum_s)
    cvscores.append((sum_s + sum_j) / 2)
    print(" ")

for i in range(fold_num):
    print((i+1),"/",fold_num)
    print("人工物画像の正解率＝",jinkou_score[i],"%")
    print("自然物画像の正解率＝",sizen_score[i],"%")
    print("全体画像の正解率＝",cvscores[i],"%")

print(" ")
print("平均人工物画像正解率：%.2f%% (+/- %.2f%%)" % (np.mean(jinkou_score), np.std(jinkou_score)))
print("平均自然物画像正解率：%.2f%% (+/- %.2f%%)" % (np.mean(sizen_score), np.std(sizen_score)))
print("平均正解率：%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Train on 1600 samples, validate on 1600 samples
Epoch 1/100
1600/1600 [==============================] - 12s 8ms/step - loss: 0.5296 - acc: 0.7662 - val_loss: 0.3255 - val_acc: 0.9372
Epoch 2/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.2461 - acc: 0.9353 - val_loss: 0.1758 - val_acc: 0.9447
Epoch 3/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.1615 - acc: 0.9416 - val_loss: 0.1393 - val_acc: 0.9572
Epoch 4/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.1407 - acc: 0.9537 - val_loss: 0.1261 - val_acc: 0.9613
Epoch 5/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.1243 - acc: 0.9578 - val_loss: 0.1142 - val_acc: 0.9641
Epoch 6/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.1167 - acc: 0.9606 - val_loss: 0.1064 - val_acc: 0.9653
Epoch 7/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.1096 - acc: 0.9622 - val_loss: 0.1028 - val_acc: 0.9669


1600/1600 [==============================] - 10s 7ms/step - loss: 0.0115 - acc: 0.9994 - val_loss: 0.0094 - val_acc: 1.0000
Epoch 61/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0107 - acc: 0.9994 - val_loss: 0.0092 - val_acc: 1.0000
Epoch 62/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0117 - acc: 0.9994 - val_loss: 0.0096 - val_acc: 1.0000
Epoch 63/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0098 - acc: 0.9997 - val_loss: 0.0084 - val_acc: 1.0000
Epoch 64/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0110 - acc: 0.9988 - val_loss: 0.0089 - val_acc: 1.0000
Epoch 65/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0094 - acc: 0.9997 - val_loss: 0.0082 - val_acc: 1.0000
Epoch 66/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0097 - acc: 0.9997 - val_loss: 0.0074 - val_acc: 1.0000
Epoch 67/100
1600/1600 [==============================

Epoch 19/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0597 - acc: 0.9841 - val_loss: 0.0554 - val_acc: 0.9841
Epoch 20/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0589 - acc: 0.9819 - val_loss: 0.0527 - val_acc: 0.9869
Epoch 21/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0572 - acc: 0.9806 - val_loss: 0.0531 - val_acc: 0.9850
Epoch 22/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0554 - acc: 0.9847 - val_loss: 0.0489 - val_acc: 0.9875
Epoch 23/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0536 - acc: 0.9856 - val_loss: 0.0491 - val_acc: 0.9872
Epoch 24/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0514 - acc: 0.9862 - val_loss: 0.0479 - val_acc: 0.9862
Epoch 25/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0502 - acc: 0.9856 - val_loss: 0.0464 - val_acc: 0.9869
Epoch 26/100
1600/1600 [=================

Epoch 79/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0094 - acc: 0.9981 - val_loss: 0.0069 - val_acc: 0.9994
Epoch 80/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0083 - acc: 0.9994 - val_loss: 0.0071 - val_acc: 0.9991
Epoch 81/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0081 - acc: 0.9988 - val_loss: 0.0064 - val_acc: 0.9994
Epoch 82/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0082 - acc: 0.9994 - val_loss: 0.0065 - val_acc: 0.9994
Epoch 83/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0076 - acc: 0.9994 - val_loss: 0.0061 - val_acc: 0.9994
Epoch 84/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0072 - acc: 0.9997 - val_loss: 0.0059 - val_acc: 0.9994
Epoch 85/100
1600/1600 [==============================] - 10s 6ms/step - loss: 0.0074 - acc: 0.9991 - val_loss: 0.0060 - val_acc: 0.9994
Epoch 86/100
1600/1600 [=================

Epoch 38/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0260 - acc: 0.9944 - val_loss: 0.0218 - val_acc: 0.9956
Epoch 39/100
1600/1600 [==============================] - 10s 7ms/step - loss: 0.0231 - acc: 0.9953 - val_loss: 0.0214 - val_acc: 0.9962
Epoch 40/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0234 - acc: 0.9941 - val_loss: 0.0203 - val_acc: 0.9962
Epoch 41/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0237 - acc: 0.9944 - val_loss: 0.0199 - val_acc: 0.9966
Epoch 42/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0212 - acc: 0.9947 - val_loss: 0.0185 - val_acc: 0.9972
Epoch 43/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0211 - acc: 0.9956 - val_loss: 0.0210 - val_acc: 0.9962
Epoch 44/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0194 - acc: 0.9975 - val_loss: 0.0171 - val_acc: 0.9975
Epoch 45/100
1600/1600 [=================

Epoch 98/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0036 - acc: 1.0000 - val_loss: 0.0030 - val_acc: 1.0000
Epoch 99/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 100/100
1600/1600 [==============================] - 11s 7ms/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 1.0000
人工物画像の正解： 387 / 402 ＝ 96.26865671641791 %
自然物画像の正解： 385 / 398 ＝ 96.73366834170855 %
 
1 / 3
人工物画像の正解率＝ 97.50623441396509 %
自然物画像の正解率＝ 97.74436090225564 %
全体画像の正解率＝ 97.62529765811036 %
2 / 3
人工物画像の正解率＝ 94.91525423728814 %
自然物画像の正解率＝ 97.9328165374677 %
全体画像の正解率＝ 96.42403538737793 %
3 / 3
人工物画像の正解率＝ 96.26865671641791 %
自然物画像の正解率＝ 96.73366834170855 %
全体画像の正解率＝ 96.50116252906324 %
 
平均人工物画像正解率：96.23% (+/- 1.06%)
平均自然物画像正解率：97.47% (+/- 0.53%)
平均正解率：96.85% (+/- 0.55%)
